# 第 11 組 HW2 程式作業

## 組員名單

- 資工三 109590010 林敬翔
- 資工三 109590011 陳彥宇
- 資工三 109590026 黃亮維

按照順序執行即可

## 6.7 Using a programming language that you are familiar with, such as C++ or Java, implement the following frequent itemset mining algorithms introduced in this chapter:

In [ ]:
# 下載 kaggle Apriori Algorithm 資料集
# https://www.kaggle.com/code/xinyidai/apriori-algorithm/data
!pip install gdown
!gdown 1PbGaP4SFbvXfaZyMVMsR17i-A3NpsAwt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1PbGaP4SFbvXfaZyMVMsR17i-A3NpsAwt
To: /content/GroceryStoreDataSet.csv
100% 478/478 [00:00<00:00, 606kB/s]


### (1) Apriori

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations, chain

In [ ]:
def powerset(s):
    return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1)))
def count_occurences(item, transactions):
    temp = 0
    for i in range(len(transactions)):
        if set(item).issubset(set(transactions[i])):
            temp += 1
    return temp

def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x:order.index(x))
    it2.sort(key=lambda x:order.index(x))

    for i in range(len(it1) - 1):
        if it1[i] != it2[i]:
            return []
    if order.index(it1[-1]) < order.index(it2[-1]):
        return it1 + [it2[-1]]
    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
        for j in range(i+1, len(set_of_its)):
            it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
            if len(it_out) > 0:
                C.append(it_out)
    return C


def get_frequent( itemsets, transactions, min_support, prev_discarded):
    LG = []
    supp_count = []
    new_discarded = []

    k = len(prev_discarded.keys())

    for s in range(len(itemsets)):
        discarded_before = False
        if k > 0:
            for it in prev_discarded[k]:
                if set(it).issubset(set(itemsets[s])):
                    discarded_before = True
                    break

        if not discarded_before:
            count = count_occurences(itemsets[s], transactions)
            if count/len(transactions) >= min_support:
                LG.append(itemsets[s])
                supp_count.append(count)
            else:
                new_discarded.append(itemsets[s])

    return LG,supp_count,new_discarded

def write_rules(X, X_S, S, conf, supp, lift, num_transactions):
    out_rules = ""
    out_rules += "Freq. Itemset: {} \n".format(X)
    out_rules += "   Rule: {} -> {} \n".format(list(S), list(X_S))
    out_rules += "   Conf: {0:2.3f} ".format(conf)
    out_rules += "   Supp: {0:2.3f} ".format(supp / num_transactions)
    out_rules += "   Lift: {0:2.3f} ".format(lift)
    out_rules += "\n"
    return out_rules

In [ ]:
df = pd.read_csv('GroceryStoreDataSet.csv',header=None)
transactions = []
for i in df[0]:
    j = (i.split(','))
    transactions.append(j)
num_trans = len(transactions)

In [ ]:
C = {}
L = {}
itemset_size = 1
discarded = {itemset_size : []}
order = []
for i in transactions:
    for j in i:
        if(j not in order):
            order.append(j)
order = sorted(order)
C.update({itemset_size : [ [f] for f in order]})
supp_count_L = {}
min_support = 0.1
min_conf = 0.1
f, sup, new_discarded = get_frequent(C[itemset_size], transactions, min_support, discarded)
discarded.update({itemset_size :  new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size : sup})


In [ ]:
k = itemset_size + 1
convergence = False
while not convergence:
    C.update({ k : join_set_itemsets(L[k-1], order)})

    f , sup, new_discarded = get_frequent(C[k], transactions, min_support, discarded)
    discarded.update({k : new_discarded})
    L.update({k : f})
    supp_count_L.update({k : sup})
    
    if len(L[k]) == 0:
        convergence = True
    k += 1


In [ ]:
assoc_rules_str =""
for i in range(1, len(L)):
    for j in range(len(L[i])):
        s = list(powerset(set(L[i][j])))
        s.pop()
        for z in s:
            S =set(z)
            X = set(L[i][j])
            X_S = set(X-S)
            sup_x = count_occurences(X, transactions)
            sup_x_s = count_occurences(X_S, transactions)
            conf = sup_x / count_occurences(S, transactions)
            lift = conf /(sup_x_s/ num_trans)
            if conf >= min_conf and sup_x >= min_support:
                assoc_rules_str += write_rules(X, X_S, S, conf, sup_x, lift, num_trans)
            

In [ ]:
# 顯示頻繁項集與關連規則
print(assoc_rules_str)

Freq. Itemset: {'BISCUIT', 'BREAD'} 
   Rule: ['BISCUIT'] -> ['BREAD'] 
   Conf: 0.571    Supp: 0.200    Lift: 0.879 
Freq. Itemset: {'BISCUIT', 'BREAD'} 
   Rule: ['BREAD'] -> ['BISCUIT'] 
   Conf: 0.308    Supp: 0.200    Lift: 0.879 
Freq. Itemset: {'BISCUIT', 'COCK'} 
   Rule: ['BISCUIT'] -> ['COCK'] 
   Conf: 0.286    Supp: 0.100    Lift: 1.905 
Freq. Itemset: {'BISCUIT', 'COCK'} 
   Rule: ['COCK'] -> ['BISCUIT'] 
   Conf: 0.667    Supp: 0.100    Lift: 1.905 
Freq. Itemset: {'BISCUIT', 'COFFEE'} 
   Rule: ['BISCUIT'] -> ['COFFEE'] 
   Conf: 0.286    Supp: 0.100    Lift: 0.714 
Freq. Itemset: {'BISCUIT', 'COFFEE'} 
   Rule: ['COFFEE'] -> ['BISCUIT'] 
   Conf: 0.250    Supp: 0.100    Lift: 0.714 
Freq. Itemset: {'BISCUIT', 'CORNFLAKES'} 
   Rule: ['BISCUIT'] -> ['CORNFLAKES'] 
   Conf: 0.429    Supp: 0.150    Lift: 1.429 
Freq. Itemset: {'BISCUIT', 'CORNFLAKES'} 
   Rule: ['CORNFLAKES'] -> ['BISCUIT'] 
   Conf: 0.500    Supp: 0.150    Lift: 1.429 
Freq. Itemset: {'BISCUIT', 'MAGGI'} 

### (2) FP-Growth

In [ ]:
from collections import defaultdict
import pandas as pd

class HeaderTable:
    def __init__(self, itemsets, min_support, frequency=None):
        self.__header_table = defaultdict(int)

        if frequency is None:
            self.frequency = [1 for i in itemsets]
        else:
            self.frequency = frequency

        for index, itemset in enumerate(itemsets):
            for item in itemset:
                self.__header_table[item] += self.frequency[index]

        self.__header_table = dict((item, support) for item, support in self.__header_table.items() if support >= min_support)

        if len(self.__header_table) == 0:
            self.__header_table = None
            return

        for item in self.__header_table:
            self.__header_table[item] = [self.__header_table[item], None]

    def __getitem__(self, key):
        return self.__header_table[key]
    
    def __contains__(self, item):
        return item in self.__header_table

    def __bool__(self):
        return self.__header_table != None
    
    def items(self):
        return self.__header_table.items()
    
    def update(self, item, target_node):
        if self.__header_table[item][1] == None:
            self.__header_table[item][1] = target_node
        else:
            current_node = self.__header_table[item][1]

            while current_node.next != None:
                current_node = current_node.next
            current_node.next = target_node

class Node:
    def __init__(self, name, frequency, parent_node):
        self.name = name
        self.count = frequency
        self.parent = parent_node
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

class FPTree:
    def __init__(self, itemsets, header_table, frequency=None):
        if frequency is None:
            self.frequency = [1 for i in itemsets]
        else:
            self.frequency = frequency

        self.__itemsets = itemsets

        self.__header_table = header_table

        self.tree = Node('Null', 1, None)
    
    def construct(self):
        if not self.__header_table:
            return
        
        for index, itemset in enumerate(self.__itemsets):
            itemset = [item for item in itemset if item in self.__header_table]
            itemset.sort(key=lambda item: self.__header_table[item][0], reverse=True)

            current_node = self.tree
            for item in itemset:
                current_node = self.__update(item, current_node, self.frequency[index])
    
    def __update(self, item, tree_node, frequency):
        if item in tree_node.children:
            tree_node.children[item].increment(frequency)
        else:
            new_node = Node(item, frequency, tree_node)
            tree_node.children[item] = new_node

            self.__header_table.update(item, new_node)
        
        return tree_node.children[item]

class FPGrowth:
    def __init__(self, dataset, min_support_ratio):
        self.__itemsets = dataset
        self.__number_of_itemsets = len(dataset)
        self.__min_support = len(dataset) * min_support_ratio
        self.__header_table = HeaderTable(dataset, self.__min_support)

        fptree = FPTree(dataset, self.__header_table)
        fptree.construct()

        self.frequent_itemsets = []
        self.__results = None
        self.__show_item_number = 0
        self.__show_name = ""

    def __ascend_fptree(self, node, prefix_path):
        if node.parent != None:
            prefix_path.append(node.name)
            self.__ascend_fptree(node.parent, prefix_path)
    
    def __find_prefix_path(self, base_path):
        tree_node = self.__header_table[base_path][1] 
        conditional_paths = []
        frequency = []

        while tree_node != None:
            prefix_path = []

            self.__ascend_fptree(tree_node, prefix_path) 

            if len(prefix_path) > 1:
                conditional_paths.append(prefix_path[1:])
                frequency.append(tree_node.count)

            tree_node = tree_node.next

        return conditional_paths, frequency

    def mine(self, prefix=set(), header_table=None):
        if header_table == None:
            header_table = self.__header_table
        
        sorted_itemsets = [item[0] for item in sorted(list(header_table.items()), key=lambda p:p[1][0])]
        
        for item in sorted_itemsets:
            new_frequent_itemset = prefix.copy()
            new_frequent_itemset.add(item)
            self.frequent_itemsets.append(new_frequent_itemset)

            conditional_pattern_base, frequency = self.__find_prefix_path(item)

            new_header_table = HeaderTable(conditional_pattern_base, self.__min_support, frequency)
            conditional_fptree = FPTree(conditional_pattern_base, new_header_table, frequency)
            conditional_fptree.construct()

            if new_header_table:
                self.mine(new_frequent_itemset, new_header_table)

    def __get_support_count(self, items):
        support = 0

        if len(items) == 1:
            item = items[0]
            support = self.__header_table[item][0]
        else:
            count = 0

            for itemset in self.__itemsets:
                if set(items).issubset(itemset):
                    count += 1
            
            support = count
        
        return support

    def __to_percentage(self, value):
        return "{:.3%}".format(value)

    def __get_column_order(self, item_number, all_letters, name=""):
        a_code = ord("A")

        column_order = []

        for index in range(item_number):
            letter = chr(a_code + index)
            column_order.append(f"{name} {letter}")

        for index in range(item_number):
            letter = chr(a_code + index)
            column_order.append(f"support_count({letter})")
            column_order.append(f"support({letter})")

        for index in range(item_number):
            letter = chr(a_code + index)
            column_order.append(f"confidence({letter})")
        
        column_order.append(f"support_count({all_letters})")
        column_order.append(f"support({all_letters})")

        return column_order
    
    def __get_results(self, item_number, name=""):
        result = {}

        a_code = ord("A")
        all_letters = ""

        for i in range(item_number):
            letter = chr(a_code + i)
            all_letters += letter + ("" if i == item_number - 1 else ", ")

            result[f"{name} {letter}"] = []
            result[f"support({letter})"] = []
            result[f"support_count({letter})"] = []
            result[f"confidence({letter})"] = []
        
        result[f"support({all_letters})"] = []
        result[f"support_count({all_letters})"] = []
        
        for itemset in self.frequent_itemsets:
            if len(itemset) == 1:
                continue
            elif len(itemset) <= item_number:
                supports = []
                for index in range(item_number):
                    letter = chr(a_code + index)

                    if index < len(itemset):
                        item = list(itemset)[index]
                        result[f"{name} {letter}"].append(item)
                        supports.append(self.__get_support_count([item]))
                        result[f"support_count({letter})"].append(supports[index])
                        result[f"support({letter})"].append(self.__to_percentage(supports[index] / self.__number_of_itemsets))
                    else:
                        result[f"{name} {letter}"].append("")
                        result[f"support({letter})"].append("")
                        result[f"support_count({letter})"].append("")
                
                support_of_itemset = self.__get_support_count(list(itemset))
                result[f"support_count({all_letters})"].append(support_of_itemset)
                result[f"support({all_letters})"].append(self.__to_percentage(support_of_itemset / self.__number_of_itemsets))

                confidences = []

                for index in range(item_number):
                    letter = chr(a_code + index)

                    if index < len(itemset):
                        support = supports[index]
                        confidence = support_of_itemset / support
                        result[f"confidence({letter})"].append(self.__to_percentage(confidence))
                        confidences.append(confidence)
                    else:
                        result[f"confidence({letter})"].append("")


        column_order = self.__get_column_order(item_number, all_letters, name)
        
        self.__results = pd.DataFrame(result).loc[:, column_order]

    def show_results(self, item_number=None, name=None):
        if self.__results is not None and item_number == None and name == None:
            return self.__results 
        if self.__results is None or item_number != self.__show_item_number or name != self.__show_name:
            self.__show_item_number = item_number
            self.__show_name = name
            self.__get_results(item_number, name)

        return self.__results

    def export_to_csv(self):
        self.__results.to_csv('result.csv')

    def __powerset(self, s):
      return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))
    
    def show_association_rule(self, min_confidence):
      rules = []
      for itemset in self.frequent_itemsets:
          subsets = self.__powerset(itemset)
          support = self.__get_support_count(list(itemset))
          for subset in subsets:
              support_of_subset = self.__get_support_count(list(subset))
              confidence = float(support / support_of_subset) if support_of_subset > 0 else 0
              if(confidence > min_confidence):
                  rules.append([set(subset), set(itemset.difference(subset)), confidence])
      return rules

In [ ]:
fp_growth = FPGrowth(transactions, 0.1)
fp_growth.mine()
fp_growth.show_results(4, "Item")

,Item A,Item B,Item C,Item D,support_count(A),support(A),support_count(B),support(B),support_count(C),support(C),support_count(D),support(D),confidence(A),confidence(B),confidence(C),confidence(D),"support_count(A, B, C, D)","support(A, B, C, D)"
0,MAGGI,JAM,,,5,25.000%,2,10.000%,,,,,40.000%,100.000%,,,2,10.000%
1,BISCUIT,MAGGI,JAM,,7,35.000%,5,25.000%,2,10.000%,,,0.000%,0.000%,0.000%,,0,0.000%
2,BISCUIT,MAGGI,JAM,TEA,7,35.000%,5,25.000%,2,10.000%,7,35.000%,0.000%,0.000%,0.000%,0.000%,0,0.000%
3,BISCUIT,MAGGI,JAM,COFFEE,7,35.000%,5,25.000%,2,10.000%,8,40.000%,0.000%,0.000%,0.000%,0.000%,0,0.000%
4,BISCUIT,MAGGI,JAM,BREAD,7,35.000%,5,25.000%,2,10.000%,13,65.000%,0.000%,0.000%,0.000%,0.000%,0,0.000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,BISCUIT,COFFEE,,,7,35.000%,8,40.000%,,,,,28.571%,25.000%,,,2,10.000%
74,BISCUIT,BREAD,COFFEE,,7,35.000%,13,65.000%,8,40.000%,,,0.000%,0.000%,0.000%,,0,0.000%
75,BISCUIT,BREAD,,,7,35.000%,13,65.000%,,,,,57.143%,30.769%,,,4,20.000%
76,TEA,BREAD,,,7,35.000%,13,65.000%,,,,,57.143%,30.769%,,,4,20.000%


In [ ]:
# 顯示頻繁項集與關連規則
fp_growth.show_association_rule(0.1)

[[{'MAGGI'}, {'JAM'}, 0.4],
 [{'JAM'}, {'MAGGI'}, 1.0],
 [{'MAGGI'}, {'BREAD', 'JAM'}, 0.4],
 [{'JAM'}, {'BREAD', 'MAGGI'}, 1.0],
 [{'BREAD'}, {'JAM', 'MAGGI'}, 0.15384615384615385],
 [{'JAM', 'MAGGI'}, {'BREAD'}, 1.0],
 [{'BREAD', 'MAGGI'}, {'JAM'}, 0.6666666666666666],
 [{'BREAD', 'JAM'}, {'MAGGI'}, 1.0],
 [{'TEA'}, {'JAM', 'MAGGI'}, 0.14285714285714285],
 [{'MAGGI'}, {'JAM', 'TEA'}, 0.2],
 [{'JAM'}, {'MAGGI', 'TEA'}, 0.5],
 [{'MAGGI', 'TEA'}, {'JAM'}, 0.25],
 [{'JAM', 'TEA'}, {'MAGGI'}, 1.0],
 [{'JAM', 'MAGGI'}, {'TEA'}, 0.5],
 [{'TEA'}, {'BREAD', 'JAM', 'MAGGI'}, 0.14285714285714285],
 [{'MAGGI'}, {'BREAD', 'JAM', 'TEA'}, 0.2],
 [{'JAM'}, {'BREAD', 'MAGGI', 'TEA'}, 0.5],
 [{'MAGGI', 'TEA'}, {'BREAD', 'JAM'}, 0.25],
 [{'JAM', 'TEA'}, {'BREAD', 'MAGGI'}, 1.0],
 [{'BREAD', 'TEA'}, {'JAM', 'MAGGI'}, 0.25],
 [{'JAM', 'MAGGI'}, {'BREAD', 'TEA'}, 0.5],
 [{'BREAD', 'MAGGI'}, {'JAM', 'TEA'}, 0.3333333333333333],
 [{'BREAD', 'JAM'}, {'MAGGI', 'TEA'}, 0.5],
 [{'JAM', 'MAGGI', 'TEA'}, {'BREAD'

### (3) Write a report to analyze the situations (e.g., data size, data distribution, minimal support threshold setting, and pattern density) where one algorithm may perform better than the others, and state why.

Apriori:
1. Apriori 採用逐層搜索的迭代方法，做法簡單，沒有復雜的理論推導，實作的難度比較低。
2. 當資料集較大時，要多次掃描資料集，因此速度會比較慢。
3. FP-Growth 會需要使用較多的記憶體空間，所以若不想使用那麼多記憶體空間可以使用 Apriori

FP-Growth:
1.	由於 FP-Growth 不需要建立候選集，因此它可以比 Apriori 算法快。
2.	當資料集過大時，由於樹的節點過多，會使用大量記憶體空間。但可以將資料集分割，使用多台電腦並行處理。
3.	當資料集較小時，由於不用像 Apriori 一樣需要多次掃描資料集，最多只需要掃描兩次就好了。因此速度較快，效率較高。
4. 隨著最小支持度的增加，兩種算法的速度都有加快，且兩者的性能差距也越來越小。因為 Apriori 能夠在每一步刪除大部分候選集並快速收斂。所以，如果想要速度更快，只要加大最小支持度就好。
5. 隨著資料量的增加，Apriori 和 FP-Growth 所需時間都會增加。但是 Apriori 比 FP-Growth 花費的時間更多。

## 6.15 The DBLP data set (www.informatik.uni-trier.de/~ley/db/) consists of over one million entries of research papers published in computer science conferences and journals. Among these entries, there are a good number of authors that have coauthor relationships.

### (a) Propose a method to efficiently mine a set of coauthor relationships that are closely correlated (e.g. often coauthoring papers together).

In [ ]:
# 下載 DBLP 資料集
!wget https://dblp.org/xml/dblp.dtd
!wget https://dblp.org/xml/dblp.xml.gz
!gzip -d dblp.xml.gz

--2022-11-07 03:49:30--  https://dblp.org/xml/dblp.dtd
Resolving dblp.org (dblp.org)... 192.76.146.204
Connecting to dblp.org (dblp.org)|192.76.146.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12622 (12K) [application/xml-dtd]
Saving to: ‘dblp.dtd’

dblp.dtd            100%[===================>]  12.33K  --.-KB/s    in 0s      

2022-11-07 03:49:32 (97.5 MB/s) - ‘dblp.dtd’ saved [12622/12622]

--2022-11-07 03:49:32--  https://dblp.org/xml/dblp.xml.gz
Resolving dblp.org (dblp.org)... 192.76.146.204
Connecting to dblp.org (dblp.org)|192.76.146.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 739940153 (706M) [application/x-gzip]
Saving to: ‘dblp.xml.gz’

dblp.xml.gz         100%[===================>] 705.66M  10.5MB/s    in 62s     

2022-11-07 03:50:35 (11.4 MB/s) - ‘dblp.xml.gz’ saved [739940153/739940153]



In [ ]:
"""
讀取 XML 並將每筆資料的作者提取出來存到 authors.txt
約需 3 分鐘
"""

import codecs
from lxml import etree

class DBLPLoader:
  def __init__(self, source_path):
    self.context = etree.iterparse(source=source_path, dtd_validation=True, load_dtd=True)
    self.current_tag = ""
  
  def clear_element(self, element):
    element.clear()

    while element.getprevious() is not None:
      del element.getparent()[0]

  def extract_authors(self, element):
    return ",".join(author.text for author in element.findall("author"))

  def parse(self, output_file):
    for _, element in self.context:
      if element.tag == "article":
        authors = self.extract_authors(element)
        output_file.write(f"{authors}\n")
      
        self.clear_element(element)

dblp_loader = DBLPLoader("dblp.xml")
output_file = codecs.open("authors.txt", "w", "utf-8")
dblp_loader.parse(output_file)
output_file.close()

In [ ]:
"""
讀取 authors.txt
並將每篇論文的作者變成一個 itemset 存到 dataset 這個二維陣列
"""

dataset = []

with open("authors.txt", "r") as file:
  for line in file.readlines():
    authors = line.strip().split(",")
    if len(authors) > 0:
      dataset.append(list(set(authors)))

In [ ]:
"""
用 FP Growth 找出經常合作寫論文的教授集合
並且顯示結果和輸出成 csv

confidence(A) 表示出現 A 教授的情況下出現 B 教授和 C 教授的信心水準
confidence(B) 表示出現 B 教授的情況下出現 A 教授和 C 教授的信心水準
confidence(C) 表示出現 C 教授的情況下出現 A 教授和 B 教授的信心水準
"""

fp_growth = FPGrowth(dataset, 0.00004)
fp_growth.mine()
fp_growth.show_results(3, "Author")

,Author A,Author B,Author C,support_count(A),support(A),support_count(B),support(B),support_count(C),support(C),confidence(A),confidence(B),confidence(C),"support_count(A, B, C)","support(A, B, C)"
0,Juan R. Torregrosa,Alicia Cordero,,138,0.005%,121,0.004%,,,86.957%,99.174%,,120,0.004%
1,A. A. Zaidan 0001,B. B. Zaidan,,125,0.004%,123,0.004%,,,96.000%,97.561%,,120,0.004%
2,Liang Gao 0001,Xinyu Li 0001,,251,0.008%,131,0.004%,,,50.199%,96.183%,,126,0.004%
3,Yongming Li 0002,Shaocheng Tong,,135,0.005%,299,0.010%,,,91.111%,41.137%,,123,0.004%
4,Richa Singh 0001,Mayank Vatsa,,138,0.005%,135,0.005%,,,96.377%,98.519%,,133,0.004%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Walid Saad,Mehdi Bennis,,519,0.017%,417,0.014%,,,24.277%,30.216%,,126,0.004%
59,Mohsen Guizani,Xiaojiang Du,,893,0.030%,427,0.014%,,,27.884%,58.314%,,249,0.008%
60,Robert Schober,Derrick Wing Kwan Ng,,721,0.024%,447,0.015%,,,20.250%,32.662%,,146,0.005%
61,Dusit Niyato,Zhu Han 0001,,710,0.024%,965,0.032%,,,17.606%,12.953%,,125,0.004%


In [ ]:
# 將結果匯出成 csv
fp_growth.export_to_csv()

### (b) Based on the mining results and the pattern evaluation measures discussed in this chapter, discuss which measure may convincingly uncover close collaboration patterns better than others

In [ ]:
class PatternEvaluation:
  def __init__(self, dataset):
    self.__itemsets = dataset

  def __count_coauthor(self, coauthor):
    result = 0
    coauthor_set = set(coauthor)

    for itemset in self.__itemsets:
      if coauthor_set.issubset(itemset):
        result += 1
    
    return result

  def __count(self, a, is_contain=True):
    result = 0

    for itemset in self.__itemsets:
      if a in itemset and is_contain:
        result += 1
      elif a not in itemset and not is_contain:
        result += 1

    return result

  def __lift(self, a, b, ab):
    return ab / (a * b)

  def __chi_square_test(self, a, b, not_a, not_b):
    total = a + b + not_a + not_b

    expected_a = (a + b) / total * a
    expected_not_a = (not_a + not_b) / total * a
    expected_b = (a + b) / total * b
    expected_not_b = (not_a + not_b) / total * b

    return (a - expected_a) ** 2 / expected_a + \
           (not_a - expected_not_a) ** 2 / expected_not_a + \
           (b - expected_b) ** 2 / expected_b + \
           (not_b - expected_not_b) ** 2 / expected_not_b

  def __all_confidence(self, a, b, ab):
    return ab / max(a, b)

  def __max_confidence(self, a, b, ab):
    return max(ab / a, ab / b)
  
  def __cosine(self, a, b, ab):
    return ab / ((a * b) ** 0.5)

  def __jaccard(self, a, b, ab):
    return ab / (a + b - ab);

  def __kulczynski(self, a, b, ab):
    return (ab / a + ab / b) * 0.5

  def show_result(self, a, b):
      number_of_coauthor = self.__count_coauthor([a, b])
      number_of_a = self.__count(a)
      number_of_not_a = self.__count(a, False)
      number_of_b = self.__count(b)
      number_of_not_b = self.__count(b, False)
      n = len(self.__itemsets)
      p_a = number_of_a / n
      p_b = number_of_b / n
      p_ab = number_of_coauthor / n

      result = {
          "Author A": [a],
          "Author B": [b],
          "support_count(A)": [number_of_a],
          "aupport_count(B)": [number_of_b],
          "Lift": [self.__lift(p_a, p_b, p_ab)],
          "X^2": [self.__chi_square_test(number_of_a, number_of_b, number_of_not_a, number_of_not_b)],
          "all confidence": [self.__all_confidence(p_a, p_b, p_ab)],
          "max confidence": [self.__max_confidence(p_a, p_b, p_ab)],
          "cosine": [self.__cosine(p_a, p_b, p_ab)],
          "jaccard": [self.__jaccard(p_a, p_b, p_ab)],
          "kulczynski": [self.__kulczynski(p_a, p_b, p_ab)],
      }

      return pd.DataFrame(result)
      
pe = PatternEvaluation(dataset)
pe.show_result("Soon Xin Ng", "Lajos Hanzo")


,Author A,Author B,support_count(A),aupport_count(B),Lift,X^2,all confidence,max confidence,cosine,jaccard,kulczynski
0,Soon Xin Ng,Lajos Hanzo,167,1175,2293.00981,6.153275e+10,0.12766,0.898204,0.338621,0.125839,0.512932


從前一題用 FP Growth 挖掘的結果我們可以發現，每個作者的論文數量只有幾百或幾千篇而已，但是 DBLP 這個資料集包含了大約 300 萬篇的論文資料，代表 null transaction 的數量非常多。如果我們是用 Lift 或 $x^2$ 來判斷的話，這兩種測量的形式會受 null trasaction 影響，在 null trasaction 數量那麼多的情況下，不適合用這兩種測量的數值來觀察，所以我們應該使用其他 null-invariant 的測量方式，像是 all confidence, max confidence, jaccard, cosine 和 kulczynski